<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/5_2_Entrenar_Word2vec_y_Doc2vec_desde_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 5 - Entrenar Word2vec y Doc2Vec desde 0

En este notebook vamos a ver cómo se entrena un modelo sencillo de word2vec eligiendo las dimensiones de los vectores.

Además, se creará un modelo de Doc2Vec a partir de un conjunto de documentos y se verá cómo se puede obtener las similitudes entre documentos.


In [1]:
# Instalamos gensim si no lo tenemos instalado
!pip3 install -U gensim
# Esto es por si no está ya instalado
!pip3 install -U pandas
!pip3 install -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.2 which is incompatible.


# Apartado 1.1 Descargamos un corpus de prueba

Vamos a probar con un corpus de noticias que se encuentra en la URL https://dis.um.es/~valencia/recursosTGINE/corpusNoticias.zip

In [2]:
# Descargamos un corpus de noticias que he creado
!wget https://dis.um.es/~valencia/recursosTGINE/corpusNoticias.zip
!unzip corpusNoticias.zip > extract.log

--2023-11-09 18:07:46--  https://dis.um.es/~valencia/recursosTGINE/corpusNoticias.zip
Resolving dis.um.es (dis.um.es)... 155.54.239.5
Connecting to dis.um.es (dis.um.es)|155.54.239.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4013278 (3.8M) [application/zip]
Saving to: ‘corpusNoticias.zip’

corpusNoticias.zip  100%[===================>]   3.83M  3.13MB/s    in 1.2s    

2023-11-09 18:07:47 (3.13 MB/s) - ‘corpusNoticias.zip’ saved [4013278/4013278]



Leemos todos lo ficheros y los metemos en una variable *texts*

Tened en cuenta que la codificación de caracteres en estos ficheros es ISO-8859-1. Esto depende del contenido de las web a descargar.

In [3]:
from os import listdir
from os.path import isfile, join

my_path = "corpusNoticias/"
texts = []
for fn in listdir(my_path):
  f = open(my_path+fn, encoding = "ISO-8859-1")
  file_content = f.read()
  texts.append(file_content)
  f.close()


# Apartado 1.2 Entrenamos un modelo word2vec a partir del corpus

Aquí vamos a entrenar un modelo word2vec con la librería GENSIM. Como Tokenizer se utilizará el word_tokenize de NLTK, pero se podría usar cualquier otro tokenizador.

In [4]:
from os import listdir
from os.path import isfile, join
import numpy
import pandas
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors
from gensim.test.utils import datapath
import re
import unicodedata
from tqdm import tqdm
import gensim
import multiprocessing
import random
from nltk.tokenize import word_tokenize


In [5]:
# Procesamos todos los textos y le aplicamos el word_tokenize de NLTK
nltk.download('punkt')
train_texts=[]
for text in texts:
     train_texts.append(word_tokenize(text.lower()))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from gensim.models import Word2Vec
# define training data
# train model
# se puede entrenar el modelo con distintos parámetros como el tamaño del vector, tamaño de la ventana, las veces que debe
# aparecer una palabra, etc.
model = Word2Vec(train_texts, vector_size=100, window=10, min_count=1, workers=10)
# summarize the loaded model
print(model)
# save model
model.save('model.bin')


Word2Vec<vocab=65297, vector_size=100, alpha=0.025>


In [7]:
# Cargamos el modelo guardado
new_model = Word2Vec.load('model.bin')

# Probamos el nuevo modelo
# Imprimimos el vector de la palabra 'energía'
print(model.wv['energía'])


[-3.32302928e-01 -2.89035082e-01  1.31270200e-01  5.38477719e-01
  1.46033362e-01 -4.52652872e-01  1.17997909e+00  9.34033573e-01
 -2.27913141e-01 -5.80577850e-01  3.51769298e-01 -3.06474417e-02
  5.70525527e-01 -4.97022510e-01 -1.37214866e-02  1.10126927e-01
  1.12488151e+00 -2.98569232e-01 -4.19529796e-01 -3.92712176e-01
  9.01951730e-01  1.03364348e+00  5.62652707e-01 -7.26178110e-01
 -4.54000324e-01 -4.38300729e-01 -2.91872285e-02  1.08416450e+00
  7.72997886e-02  4.64841068e-01  1.24063981e+00 -6.61283553e-01
 -7.46721208e-01 -4.57586735e-01 -6.43015146e-01  1.24790418e+00
  9.05629575e-01  3.25669765e-01 -4.25543457e-01 -4.89855886e-01
  1.06388533e+00 -7.42860258e-01 -4.94016737e-01  1.07361056e-01
  3.62946540e-01  6.03122890e-01 -5.73800206e-01 -6.38415694e-01
  9.11710441e-01  2.71047741e-01  1.78090081e-01 -8.88756633e-01
  2.20009759e-01 -2.53252447e-01 -9.13177356e-02  6.42076492e-01
  6.93647087e-01 -4.05200005e-01  6.99490428e-01 -3.32431436e-01
 -1.34559870e-01 -4.71104

In [22]:
# Probamos similitudes
new_model.wv.similarity("jesus", "fracasado")

0.26892635

In [9]:
# Probamos en listar alguna de las palabras más similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'energía'
palabra = 'murcia'
print(new_model.wv.most_similar(palabra))

[('covid-19', 0.9623693823814392), ('coronavirus', 0.9595177173614502), ('contagio', 0.8973839282989502), ('sars-cov-2', 0.8792991042137146), ('casos', 0.8700515031814575), ('vacunación', 0.8673209547996521), ('contagios', 0.8666442036628723), ('pfizer', 0.8611642122268677), ('datos', 0.860546350479126), ('dosis', 0.8578240871429443)]
[('universidad', 0.9563602209091187), ('mancha', 0.9304651021957397), ('castilla-la', 0.9282267689704895), ('emergencias', 0.9267937541007996), ('geográfico', 0.9201173186302185), ('microbiología', 0.9180510640144348), ('instituto', 0.9114208817481995), ('extremadura', 0.9081405997276306), ('gomera', 0.9078753590583801), ('aragón', 0.9056111574172974)]


In [10]:
# Probamos alguna analogía
# Covid es a Vacunas lo que Salud es a ...
print(new_model.wv.most_similar(positive=["salud", "vacunas"], negative=["covid"], topn=10))

# Covid es a Vacunas lo que Guerra es a ...
print(new_model.wv.most_similar(positive=["guerra", "vacunas"], negative=["covid"], topn=10))


[('pública', 0.8335195183753967), ('sanidad', 0.8143428564071655), ('medidas', 0.8075898885726929), ('ema', 0.8017637133598328), ('seguridad', 0.7991828918457031), ('variantes', 0.7895002961158752), ('oms', 0.789139986038208), ('prevención', 0.7885681986808777), ('comunidades', 0.7867915630340576), ('administración', 0.7798408269882202)]
[('importancia', 0.8592158555984497), ('opinión', 0.854713499546051), ('democracia', 0.8495200872421265), ('respuesta', 0.8483499884605408), ('estrategia', 0.8472895622253418), ('crisis', 0.8432221412658691), ('postura', 0.8402270674705505), ('cuestión', 0.8377296924591064), ('política', 0.8322826027870178), ('voluntad', 0.8271920680999756)]


In [11]:
# Probamos a mostrar términos similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'ucrania'
palabra = 'ucrania'
print(new_model.wv.most_similar(palabra))

[('covid-19', 0.9623693823814392), ('coronavirus', 0.9595177173614502), ('contagio', 0.8973839282989502), ('sars-cov-2', 0.8792991042137146), ('casos', 0.8700515031814575), ('vacunación', 0.8673209547996521), ('contagios', 0.8666442036628723), ('pfizer', 0.8611642122268677), ('datos', 0.860546350479126), ('dosis', 0.8578240871429443)]
[('invasión', 0.9214394688606262), ('económica', 0.9032465219497681), ('bruselas', 0.9024725556373596), ('guerra', 0.8988350033760071), ('estrategia', 0.897479772567749), ('prioridad', 0.8882173299789429), ('importancia', 0.8842669129371643), ('rusia', 0.8834152817726135), ('juventud', 0.8769779801368713), ('ponencia', 0.8745707273483276)]


# Apartado 1.3 Entrenamos un Doc2Vec con los mismos textos


In [23]:
#Import all the dependencies
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

#Necestiamos crear un TaggedDocument para cada uno de los textos indicando un índice de cada texto
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(train_texts)]


In [24]:
# Definimos los parámetros de entrenamiento y entrenamos
max_epochs = 10
vec_size = 50
alpha = 0.025

doc2vec_model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm = 1,
                epochs = max_epochs)

doc2vec_model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    doc2vec_model.train(tagged_data,
                total_examples=doc2vec_model.corpus_count,
                epochs=doc2vec_model.epochs)
    # decrease the learning rate
    doc2vec_model.alpha -= 0.0002
    # fix the learning rate, no decay
    doc2vec_model.min_alpha = model.alpha

doc2vec_model.save("d2v.model")
print("Model Saved")

Model Saved


In [26]:
from gensim.models import Doc2Vec

doc2vec_model= Doc2Vec.load("d2v.model")
# Probamos a encontrar textos similares a uno dado
query_text = "españa"
test_data = word_tokenize(query_text)
v1 = doc2vec_model.infer_vector(test_data)

# Encontramos los documentos más similares
similar_doc = doc2vec_model.dv.most_similar(v1)

# Imprimimos los 5 documentos más similares
top5_similar_doc = similar_doc[:5]
print(top5_similar_doc)
for doc in top5_similar_doc:
  print("--------------------------")
  print(texts[int(doc[0])])
  print('Similitud:',doc[1])



[('1341', 0.6503051519393921), ('1426', 0.62728351354599), ('1211', 0.6272633671760559), ('189', 0.6272565126419067), ('544', 0.625141441822052)]
--------------------------
Varios dirigentes del PPC, increpados en Vic cuando celebraban un acto público
El presidente del PPC, Alejandro Fernández, y otros dirigentes del partido han sido recibidos este sábado en el centro de Vic (Barcelona) con insultos, abucheos y gritos de "fascistas" por un grupo de manifestantes, en una tensa protesta que ha obligado a los Mossos a abrir paso a los políticos. Los dirigentes populares habían convocado este mediodía una rueda de prensa en la plaza Mayor de Vic, donde el partido pidió permisos para montar una carpa informativa que el Ayuntamiento de la población les ha denegado. Cuando el grupo de políticos caminaba por el centro de Vic, decenas de personas se han concentrado para impedirles el paso, mientras les increpaban a gritos de "fascistas" o "fuera, fuera" y proferían insultos. En la protesta se h